In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import subprocess

import regex as re
from packaging import version


In [4]:
# Run this to check if SLEAP is the new or old version on the device.

# This checks the conda version of the SLEAP being used since apparently they changed the confidence of a point from .p to .score.
# Potential Issue: I haven't checked if this works on old SLEAP envrionmnts 
# result = subprocess.run(
#     ['conda', 'run', '-n', 'sleap', 'python', '-c', 'import sleap; sleap.versions()'],
#     capture_output=True,
#     text=True
# )
# print(type(result.stdout))



# match = re.search(r'SLEAP:\s*([\d.]+)', result.stdout)
# if match:
#     sleap_version = match.group(1)
#     print(sleap_version)  # 1.5.2

# if match:
#     sleap_version = match.group(1)
    
#     # Compare versions
#     if version.parse(sleap_version) >= version.parse("1.4.1"):
#         print(f"SLEAP {sleap_version} is >= 1.4.1")
#         SLEAP_NEW = True  
#     else:
#         print(f"SLEAP {sleap_version} is < 1.4.1")
#         SLEAP_OLD = True  

In [5]:
# if SLEAP_NEW:
#     body_coords = ['.x','.y','.score']
# else:
#     body_coords = ['.x','.y','.p']

In [6]:
## This is a search script that will find the files that match a pattern in the directory and return said matches

# Rootdir= "C:/MolasLAB/simba_training_videos/Predictions_file/unclean_predictions" # replace this the scope of the search
# OutputPath="C:/MolasLAB/simba_training_videos/Predictions_file/cleaned_predictions" # os.getcwd() # can be replaced with a dedicated path ie 'C:/Users/Jume6823.AD.000/Downloads'

# Rootdir= "C:/MolasLAB/simba_training_videos/Predictions_file/unclean_predictions" # replace this the scope of the search
Rootdir =r"C:\Users\jjmmc\Documents\no_cleaning"
ALL_FILES_IN_ROOT = True
#SinglePattern= "L\d{1}\sTrap2\d{3}.analysis$"
#PairPattern = "L\d{1}\sTrap2\d{3}[+]\d{3}.analysis$"

def CSVSearch(RootDir, pair:bool,  pattern_on  = False):
    if pattern_on == True:
        if pair == False:
            pattern= re.compile(r'L\d{1}\sTrap2\d{3}.analysis')
        else:
            pattern= re.compile(r'L\d{1}\sTrap2\d{3}[-+]\d{3}.analysis')
    
    matching = []
    for item in os.listdir(RootDir):
        filepath = os.path.join(RootDir, item).replace("\\", "/")
        if os.path.isfile(filepath):  # Only include files, not directories
            matching.append(filepath)
    return(matching)

In [7]:

if ALL_FILES_IN_ROOT:
    Doubles = CSVSearch(Rootdir, False)
# Doubles= [r"C:\Users\jjmmc\Downloads\simple_social_test_csv.csv"]

print(Doubles)


['C:/Users/jjmmc/Documents/no_cleaning/20250116_TRAP2_5VLS_social_L4_TRAP2_205+206.csv', 'C:/Users/jjmmc/Documents/no_cleaning/20250116_TRAP2_5VLS_social_L4_TRAP2_208+209.csv', 'C:/Users/jjmmc/Documents/no_cleaning/20250116_TRAP2_5VLS_social_L4_TRAP2_218+219.csv', 'C:/Users/jjmmc/Documents/no_cleaning/20250117_TRAP2_5VLS_social_L5_TRAP2_205+206.csv', 'C:/Users/jjmmc/Documents/no_cleaning/20250117_TRAP2_5VLS_social_L5_TRAP2_208+209.csv', 'C:/Users/jjmmc/Documents/no_cleaning/20250117_TRAP2_5VLS_social_L5_TRAP2_218+219.csv']


In [8]:
body_coords = ['.x','.y','.score']

def identity_swap_tracker(animal_1_df, animal_2_df, body_parts):
    """Detect potential identity swaps between two tracked animals based on body part positions."""
    shifted_1 = animal_1_df.shift(1).bfill()
    shifted_2 = animal_2_df.shift(1).bfill()

    summed_euclidean_1 = np.zeros(len(animal_1_df))
    summed_euclidean_2 = np.zeros(len(animal_1_df))
    summed_euclidean_1_to_2 = np.zeros(len(animal_1_df))
    summed_euclidean_2_to_1 = np.zeros(len(animal_1_df))

    for part in body_parts:
        animal1_coords = animal_1_df.filter(like=part).to_numpy()
        animal2_coords = animal_2_df.filter(like=part).to_numpy()
        shifted1_coords = shifted_1.filter(like=part).to_numpy()
        shifted2_coords = shifted_2.filter(like=part).to_numpy()

        dist_1 = np.linalg.norm(shifted1_coords - animal1_coords, axis=1)
        dist_2 = np.linalg.norm(shifted2_coords - animal2_coords, axis=1)
        dist_1_to_2 = np.linalg.norm(shifted1_coords - animal2_coords, axis=1)
        dist_2_to_1 = np.linalg.norm(shifted2_coords - animal1_coords, axis=1)

        summed_euclidean_1 += dist_1
        summed_euclidean_2 += dist_2
        summed_euclidean_1_to_2 += dist_1_to_2
        summed_euclidean_2_to_1 += dist_2_to_1
    potential_swap_1_to_2 = summed_euclidean_1_to_2 < summed_euclidean_1
    potential_swap_2_to_1 = summed_euclidean_2_to_1 < summed_euclidean_2

    swapped_indices = np.where(potential_swap_1_to_2 & potential_swap_2_to_1)[0] 
    return swapped_indices


def collision_detector(df0, df1, body_parts, pairwise_threshold=10, continuity_threshold=50):
    """
    This detects if two skeletons are too close and therefore not distinct which can cause big issues for interpolation.
    Notably this uses the set of pairwise differences to determine this with the criteria being if the two seketons effectively share 3 or. This is since in ideal pose data one can have 1-2 parts reasonably be in vey close proximity (ie the noses or the ears)
    This returns the dataframes with these problematic frames removed. 
    """

    df0 = df0.copy().reset_index(drop=True)
    df1 = df1.copy().reset_index(drop=True)
    N = len(df0)

    # 1. Convert the pandas data to numpy
    
    # For each bodypart create vectors of shape (N, num_parts*2)
    coords0 = np.column_stack([df0[f"{bp}.x"].to_numpy() for bp in body_parts] +
                              [df0[f"{bp}.y"].to_numpy() for bp in body_parts])
    coords1 = np.column_stack([df1[f"{bp}.x"].to_numpy() for bp in body_parts] +
                              [df1[f"{bp}.y"].to_numpy() for bp in body_parts])

    num_parts = len(body_parts)

    # Split x/y back out for pairwise distance operations
    x0 = coords0[:, :num_parts]
    y0 = coords0[:, num_parts:]
    x1 = coords1[:, :num_parts]
    y1 = coords1[:, num_parts:]

    # 2. Calculate the distance between identites with respect to the same frame
    pairwise_dist = np.sqrt((x0 - x1)**2 + (y0 - y1)**2)   # shape (N, num_parts)

    # Count parts whose distances exceed threshold
    separated_count = np.nansum(pairwise_dist > pairwise_threshold, axis=1)

    # Collision mask: < 3 separated bp
    coll_mask = separated_count < 3
    coll_mask[0] = False  # first frame cannot be evaluated

    # 3. Compure the framewise differences of the identities, 

    # Shifted (t-1) frames
    x0_prev = np.roll(x0, 1, axis=0)
    y0_prev = np.roll(y0, 1, axis=0)
    x1_prev = np.roll(x1, 1, axis=0)
    y1_prev = np.roll(y1, 1, axis=0)

    # This then corrects the wrapping that comes as a result of rolling
    x0_prev[0] = x0_prev[1]
    y0_prev[0] = y0_prev[1]
    x1_prev[0] = x1_prev[1]
    y1_prev[0] = y1_prev[1]

    # Euclidean norm summed across parts
    def partwise_dist(ax, ay, bx, by):
        return np.nansum(np.sqrt((ax - bx)**2 + (ay - by)**2), axis=1) #nansum accounts for missing data as treated it as 0 

    d00 = partwise_dist(x0, y0, x0_prev, y0_prev)
    d11 = partwise_dist(x1, y1, x1_prev, y1_prev)
    # d01 = partwise_dist(x0, y0, x1_prev, y1_prev)
    # d10 = partwise_dist(x1, y1, x0_prev, y0_prev)

    # Continuity masks
    cont00_bad = d00 > continuity_threshold
    cont11_bad = d11 > continuity_threshold
    # cont01_bad = d01 > continuity_threshold
    # cont10_bad = d10 > continuity_threshold

    
    # 4. Condtions for identity/frame removal
    
    # Remove both if both identity and swapped continuity fails.
    remove_both = (cont00_bad & cont11_bad & coll_mask)
    
    # remove_both = (cont00_bad & cont11_bad & cont01_bad & cont10_bad & coll_mask)

    #  Otherwise remove least continuous identity 
    #     Compare identity vs swapped continuity scores.
    #     Use raw distances to pick the larger (= worse).
    
    # cont_id0 = np.minimum(d00, d01)   # best mapping for ID0
    # cont_id1 = np.minimum(d11, d10)   # best mapping for ID1

    remove_0 = (d00 > d11) & coll_mask & (~remove_both)
    remove_1 = (d11 > d00) & coll_mask & (~remove_both)
    # remove_0 = (cont_id0 > cont_id1) & coll_mask & (~remove_both)
    # remove_1 = (cont_id1 > cont_id0) & coll_mask & (~remove_both)
    removed_indices = {
        'identity_0': np.where(remove_0)[0].tolist(),
        'identity_1': np.where(remove_1)[0].tolist(),
        'both': np.where(remove_both)[0].tolist()
    }
    # 5. Remove the bad identites 
    cols = [f"{bp}{ax}" for bp in body_parts for ax in body_coords]

    df0.loc[remove_0 | remove_both, cols] = np.nan
    df1.loc[remove_1 | remove_both, cols] = np.nan
  
    return df0, df1, removed_indices

def gap_identity_tracker(df0, df1, body_parts):
    """
    Detects potential swaps over gaps where one or both identities are missing.
    Returns a list of (start_frame, end_frame) tuples where a swap should be applied.
    """

    all_frames = df0['frame_idx'].to_numpy()
    exist0 = df0.dropna(subset=[f"{bp}.x" for bp in body_parts])['frame_idx'].to_numpy()
    exist1 = df1.dropna(subset=[f"{bp}.x" for bp in body_parts])['frame_idx'].to_numpy()

    missing_both_mask = ~(np.isin(all_frames, exist0) | np.isin(all_frames, exist1))
    swapped_intervals = []

    # Split missing frames into consecutive gaps
    double_gaps = np.split(all_frames[missing_both_mask], np.where(np.diff(all_frames[missing_both_mask]) != 1)[0]+1)
    for gap in double_gaps:
        if len(gap) == 0:
            continue

        frame_start = gap[0] - 1
        frame_end = gap[-1] + 1

        # Get last known positions before the gap
        last0 = df0[df0['frame_idx'] <= frame_start].tail(1)
        last1 = df1[df1['frame_idx'] <= frame_start].tail(1)

        # Get first known positions after the gap
        next0 = df0[df0['frame_idx'] >= frame_end].head(1)
        next1 = df1[df1['frame_idx'] >= frame_end].head(1)

        if last0.empty or last1.empty or next0.empty or next1.empty:
            continue  # Can't decide swap if boundary data missing

        # Calculate distances if identities are swapped
        dist_no_swap = 0
        dist_swap = 0
        for bp in body_parts:
            for suffix in ['.x', '.y']:
                last0_val = last0[f"{bp}{suffix}"].values[0]
                last1_val = last1[f"{bp}{suffix}"].values[0]
                next0_val = next0[f"{bp}{suffix}"].values[0]
                next1_val = next1[f"{bp}{suffix}"].values[0]

                dist_no_swap += (next0_val - last0_val)**2 + (next1_val - last1_val)**2
                dist_swap += (next0_val - last1_val)**2 + (next1_val - last0_val)**2

        if dist_swap < dist_no_swap:
            # store the last frame 
            swapped_intervals.append( gap[-1])
    print(swapped_intervals)
    return swapped_intervals


def apply_cumulative_swaps(df0, df1, swap_points):
    """
    Apply cumulative identity swaps between two animal tracking DataFrames.
    Each swap point indicates the frame index *after which* identities flip.
    Swapping twice restores the original identity, so only the parity matters.
    """
    df0 = df0.copy()
    df1 = df1.copy()

    # Sort and deduplicate swap points
    swap_points = np.sort(np.unique(swap_points))
    if len(swap_points) == 0:
        return df0, df1

    frames = df0['frame_idx'].to_numpy()
    # Determine whether each frame should be flipped (odd number of swaps so far)
    flip_mask = np.cumsum(np.isin(frames, swap_points)) % 2 == 1
    
    # Get columns to swap (exclude frame_idx and track)
    cols_to_swap = [col for col in df0.columns if col not in ['frame_idx', 'track']]
    
    # Apply swap only to coordinate columns
    temp = df0.loc[flip_mask, cols_to_swap].copy()
    df0.loc[flip_mask, cols_to_swap] = df1.loc[flip_mask, cols_to_swap].values
    df1.loc[flip_mask, cols_to_swap] = temp.values

    return df0, df1



def double_identity_tracker(index):
    """
    Reads tracking data from Doubles[index], separates by identity,
    detects and corrects identity swaps (both normal and gap-based),
    interpolates missing data, and saves the cleaned dataset.
    """

    #  Step 1: Load CSV and change the columns if from a newer version of SLEAP
    df = pd.read_csv(Doubles[index], delimiter=',', quotechar='|')
    df.rename(columns=lambda x: x.replace('.p', '.score'), inplace=True)
    print(f"Loaded {len(df)} rows from {Doubles[index]}")

    #  Step 2: Parse IDs and frame numbers 
    df['track'] = df['track'].astype(str).str.extract(r'(\d+)$').astype(int)
    df['frame_idx'] = df['frame_idx'].astype(int)
    df = df.apply(pd.to_numeric, errors='ignore')

    #  Step 3: Define frame range 
    min_frame = 0
    max_frame = df['frame_idx'].iloc[-1]
    frame_range = np.arange(min_frame, max_frame + 1)

    #  Step 4: Identify body parts and coordinates
    body_parts = ["nose", "ear_left", "ear_right", "head", "body_center", "tail_base"]
    
    #  Step 5: Split into identity DataFrames 
    identities = sorted(df['track'].unique())
    if len(identities) != 2:
        print(identities)
        raise ValueError("This function currently supports exactly two identities.")
        

    tracks = {}
    for tid in identities:
        tdf = df[df['track'] == tid].set_index('frame_idx').reindex(frame_range).reset_index()
        tdf['track'] = tid
        tracks[tid] = tdf

    print(f"Created {len(tracks)} identity tracks with {len(frame_range)} frames each")
    
    df0, df1 = tracks[identities[0]], tracks[identities[1]]
    # Step 5.5: Remove overlapped identities
    df0, df1, collisions = collision_detector(df0, df1, body_parts)
    # Step 6: Detect gap swaps and fix BEFORE interpolation 
    print("Detecting gap-based identity swaps...")
    swap_gaps = gap_identity_tracker(df0, df1, body_parts)  # returns list of intervals [(start, end), ...]
    df0, df1 = apply_cumulative_swaps(df0, df1, swap_gaps)

    # Step 7: Interpolate missing coordinates AFTER gap swaps 
    for tdf in [df0, df1]:
        for bp in body_parts:
            for suffix in body_coords:
                col = next((c for c in tdf.columns if bp + suffix in c), None)
                if col and col in tdf.columns:
                    tdf[col] = tdf[col].interpolate(limit_direction='both').astype(float)
    df0['track'],df1['track']= df0['track'][0], df1['track'][0] # This is fixing the track identities so that they are interpreted properly by the next function

    
    #  Step 8: Detect gap swaps and fix AFTER interpolation 

    print("Detecting normal identity swaps...")
    swap_normal = identity_swap_tracker(df0, df1, body_parts)  # returns individual frame indices
    
    df0, df1 = apply_cumulative_swaps(df0, df1, swap_normal)
    df0['track'],df1['track']= df0['track'][0], df1['track'][0]
    all_swaps_frames = np.sort(np.unique(np.concatenate([swap_normal] + 
                                                        [swap_gaps])))
    print(f"Total swaps detected: {len(all_swaps_frames)} at frames: {all_swaps_frames}")

    # df0, df1 = apply_cumulative_swaps(df0, df1, all_swaps_frames)

    #  Step 9: Recombine and export 

    df_clean = pd.concat([df0, df1], ignore_index=True).sort_values(['frame_idx', 'track'])
    # Move track to first position
    df_clean.insert(0, 'track', df_clean.pop('track'))
    
    out_name = os.path.basename(Doubles[index])
    
    processed_path = os.path.join(Rootdir,'processed')
    
    if not os.path.exists(processed_path):
        os.makedirs(processed_path)
    
    full_path = os.path.join(processed_path, out_name)
    df_clean.to_csv(full_path, index=False)
    
    print(f"Cleaned file saved as {out_name}")
    
    #  Step 10: Return swap info 

    swap_path =os.path.join(Rootdir,'swap_data')
    
    if not os.path.exists(swap_path):
        os.makedirs(swap_path)
    
    df_swaps = pd.DataFrame({
        'swap_gaps': pd.Series(swap_gaps),
        'swap_normal': pd.Series(swap_normal),
        'collisions_id1': pd.Series(collisions['identity_0']),
        'collisions_id2': pd.Series(collisions['identity_1']),
        'collisions_both': pd.Series(collisions['both'])
    })
    
    print(df_swaps)
    full_swap_path= os.path.join(swap_path, f'{out_name}')
    df_swaps.to_csv(full_swap_path, index=False)
    
    return {
        "swapped_frames": all_swaps_frames
    }


In [9]:

# double_identity_tracker(5)
for i in range(len(Doubles)):
    double_identity_tracker(i)

Loaded 116113 rows from C:/Users/jjmmc/Documents/no_cleaning/20250116_TRAP2_5VLS_social_L4_TRAP2_205+206.csv


C:\Users\jjmmc\AppData\Local\Temp\ipykernel_71404\3979579838.py:231: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


Created 2 identity tracks with 62689 frames each
Detecting gap-based identity swaps...
[np.int64(2), np.int64(2473), np.int64(2789), np.int64(5294)]
Detecting normal identity swaps...
Total swaps detected: 28 at frames: [    2  2191  2473  2789  4435  4769  4836  4837  5294  6450  6986 10996
 11622 11624 11626 17073 19494 32011 39383 40474 42628 43291 43515 45422
 48527 48534 56320 59476]
Cleaned file saved as 20250116_TRAP2_5VLS_social_L4_TRAP2_205+206.csv
      swap_gaps  swap_normal  collisions_id1  collisions_id2  collisions_both
0           2.0       2191.0               1            48.0           2756.0
1        2473.0       4435.0               2            50.0           2757.0
2        2789.0       4769.0             122            51.0           7791.0
3        5294.0       4836.0             968            54.0           9846.0
4           NaN       4837.0             969            56.0          26259.0
...         ...          ...             ...             ...          

C:\Users\jjmmc\AppData\Local\Temp\ipykernel_71404\3979579838.py:231: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


Created 2 identity tracks with 62558 frames each
Detecting gap-based identity swaps...
[np.int64(1372), np.int64(5581), np.int64(6253), np.int64(13345), np.int64(22813), np.int64(30119), np.int64(35319), np.int64(35517), np.int64(35758), np.int64(41398), np.int64(55373)]
Detecting normal identity swaps...
Total swaps detected: 36 at frames: [  813   989  1086  1372  3163  3509  3619  5018  5581  5599  5722  5738
  5902  6230  6253  6579 11716 13345 15611 22813 29756 29761 30119 30261
 35319 35517 35758 41378 41381 41398 44485 44538 55373 55375 55376 61841]
Cleaned file saved as 20250116_TRAP2_5VLS_social_L4_TRAP2_208+209.csv
      swap_gaps  swap_normal  collisions_id1  collisions_id2  collisions_both
0        1372.0        813.0           433.0              66           1314.0
1        5581.0        989.0           434.0              78           1322.0
2        6253.0       1086.0           435.0              79           6233.0
3       13345.0       3163.0           436.0           

...

Created 2 identity tracks with 34346 frames each
Detecting gap-based identity swaps...
[np.int64(5679), np.int64(10030), np.int64(32781)]
Detecting normal identity swaps...
Total swaps detected: 34 at frames: [  452   456  2963  2965  3787  3821  3936  5679  8699  8700  9424 10030
 10051 11494 11495 13240 15485 15486 17103 17158 17232 17265 17284 17286
 17288 17289 21849 22196 22207 22208 22209 22674 32779 32781]
Cleaned file saved as 20250117_TRAP2_5VLS_social_L5_TRAP2_205+206.csv
      swap_gaps  swap_normal  collisions_id1  collisions_id2  collisions_both
0        5679.0        452.0           446.0              30           1885.0
1       10030.0        456.0           447.0              33           5608.0
2       32781.0       2963.0           448.0              36           5609.0
3           NaN       2965.0           449.0             239           9420.0
4           NaN       3787.0           450.0             251          11496.0
...         ...          ...             ... 

C:\Users\jjmmc\AppData\Local\Temp\ipykernel_71404\3979579838.py:231: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


Created 2 identity tracks with 36210 frames each
Detecting gap-based identity swaps...
[np.int64(934), np.int64(940), np.int64(7219), np.int64(8091), np.int64(21281), np.int64(33625)]
Detecting normal identity swaps...
Total swaps detected: 36 at frames: [  934   937   938   939   940  1941  4030  4044  4228  7219  7915  7923
  8091  8397  8692 10448 10479 11577 11579 15231 15233 15234 15254 19473
 20674 21281 21287 21299 21495 21496 22125 27531 31281 33625 33672 33990]
Cleaned file saved as 20250117_TRAP2_5VLS_social_L5_TRAP2_208+209.csv
      swap_gaps  swap_normal  collisions_id1  collisions_id2  collisions_both
0         934.0        937.0             1.0             139           8253.0
1         940.0        938.0             5.0             164           8254.0
2        7219.0        939.0             6.0             165          11276.0
3        8091.0       1941.0             7.0             166          17962.0
4       21281.0       4030.0             8.0             167     

C:\Users\jjmmc\AppData\Local\Temp\ipykernel_71404\3979579838.py:231: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


Detecting gap-based identity swaps...
[np.int64(2502), np.int64(9935), np.int64(23454), np.int64(36404)]
Detecting normal identity swaps...
Total swaps detected: 30 at frames: [ 1938  1939  1940  2094  2502  2504  5704  5744  6839  6852  7562  9935
 12501 13897 14966 14968 14980 15916 16762 23448 23453 23454 25320 25322
 25328 25329 35036 36233 36404 38539]
Cleaned file saved as 20250117_TRAP2_5VLS_social_L5_TRAP2_218+219.csv
      swap_gaps  swap_normal  collisions_id1  collisions_id2  collisions_both
0        2502.0       1938.0            53.0             120           7106.0
1        9935.0       1939.0            54.0             122           7109.0
2       23454.0       1940.0            55.0             123           7703.0
3       36404.0       2094.0            56.0             124          26252.0
4           NaN       2504.0           341.0             125              NaN
...         ...          ...             ...             ...              ...
1635        NaN         